In [42]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer

import pandas as pd
import numpy as np

In [57]:
data = pd.read_excel(r"C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\KRP-CKPN\2025-05\Test KRP RB per product\Data for MEV model.xlsx",
                    sheet_name = 'forModel')

In [58]:
data.head()

,No,Yr-mth,Inflation,InflationDelta,BIRate,BIRateDelta,BIRateDeltaL2,GDP,GDPL1,BIRateChangeL3,...,NonWODPD%-KMG,NonWODPD%-KMK,NonWODPD%-KPM,NonWODPD%-KPR,NonWODPD%-PKP,ORDNPL-KMG,ORDNPL-KMK,ORDNPL-KPM,ORDNPL-KPR,ORDNPL-PKP
0,1,2022-12,5.5,-0.5,5.50,1.25,2.00,5.000000,5.730000,0.00,...,0.023438,0.007018,0.007259,0.040892,0.036290,0.022901,0.022298,0.001672,0.021818,0.015873
1,2,2023-01,5.3,-0.4,5.75,1.00,2.25,5.013333,5.486667,0.00,...,0.015748,0.012433,0.007108,0.029963,0.032258,0.023077,0.022569,0.001638,0.021978,0.015873
2,3,2023-02,5.5,0.1,5.75,0.50,2.00,5.026667,5.243333,0.25,...,0.023622,0.008929,0.008796,0.045455,0.036000,0.023077,0.022688,0.001647,0.022222,0.015748
3,4,2023-03,5.0,-0.5,5.75,0.25,1.50,5.040000,5.000000,0.75,...,0.023622,0.003597,0.007159,0.042802,0.016393,0.007812,0.022847,0.001649,0.026616,0.012146
4,5,2023-04,4.3,-1.0,5.75,0.00,1.00,5.083333,5.013333,1.25,...,0.023622,0.007260,0.012121,0.051181,0.024590,0.007812,0.023050,0.001650,0.026923,0.012146


In [60]:
data.dtypes

No                     int64
Yr-mth                object
Inflation            float64
InflationDelta       float64
BIRate               float64
BIRateDelta          float64
BIRateDeltaL2        float64
GDP                  float64
GDPL1                float64
BIRateChangeL3       float64
InflationChange      float64
BIRateL4             float64
GDPDelta             float64
GDPDeltaL2           float64
InflationChangeL1    float64
GDPChange            float64
GDPChangeL4          float64
NonWODPD%-KMG        float64
NonWODPD%-KMK        float64
NonWODPD%-KPM        float64
NonWODPD%-KPR        float64
NonWODPD%-PKP        float64
ORDNPL-KMG           float64
ORDNPL-KMK           float64
ORDNPL-KPM           float64
ORDNPL-KPR           float64
ORDNPL-PKP           float64
dtype: object

In [73]:
X = data[['InflationDelta', 'BIRateDelta', 'BIRateDeltaL2', 'GDPL1', 'BIRateChangeL3',
        'BIRateL4', 'GDPDelta', 'GDPDeltaL2', 'InflationChangeL1', 'GDPChange', 'GDPChangeL4']]
y = data['ORDNPL-PKP']

In [74]:
log_transformer = FunctionTransformer(np.log, validate=True)
# X_log = log_transformer.fit_transform(X)

# Alternatively, transform dependent variable y:
y_log = np.log(y)

# Initialize the linear regression model
lr = LinearRegression()

# Initialize SequentialFeatureSelector for forward selection
# k_features='best' selects the best number of features
# forward=True for forward selection, False for backward elimination
# scoring='r2' uses R-squared as the evaluation metric
sfs = SFS(lr, 
          k_features='best', 
          forward=True, 
          floating=False, 
          scoring='r2',
          cv=0) # cv=0 for no cross-validation, or specify number of folds

# Fit SFS to the data
sfs.fit(X, y_log)

# Get the selected features
selected_features = list(sfs.k_feature_names_)
print("Selected features:", selected_features)

df_SFS_results = pd.DataFrame(sfs.subsets_).transpose()
df_SFS_results

Selected features: ['InflationDelta', 'BIRateDelta', 'BIRateDeltaL2', 'GDPL1', 'BIRateChangeL3', 'BIRateL4', 'GDPDelta', 'GDPDeltaL2', 'InflationChangeL1', 'GDPChange', 'GDPChangeL4']


,feature_idx,cv_scores,avg_score,feature_names
1,"(5,)",[0.7774988407347124],0.777499,"(BIRateL4,)"
2,"(5, 6)",[0.817280103393983],0.81728,"(BIRateL4, GDPDelta)"
3,"(4, 5, 6)",[0.8496712078343182],0.849671,"(BIRateChangeL3, BIRateL4, GDPDelta)"
4,"(4, 5, 6, 9)",[0.8677492956350987],0.867749,"(BIRateChangeL3, BIRateL4, GDPDelta, GDPChange)"
5,"(4, 5, 6, 7, 9)",[0.8726792694500553],0.872679,"(BIRateChangeL3, BIRateL4, GDPDelta, GDPDeltaL..."
6,"(1, 4, 5, 6, 7, 9)",[0.87805663645309],0.878057,"(BIRateDelta, BIRateChangeL3, BIRateL4, GDPDel..."
7,"(1, 2, 4, 5, 6, 7, 9)",[0.8935922036682129],0.893592,"(BIRateDelta, BIRateDeltaL2, BIRateChangeL3, B..."
8,"(1, 2, 4, 5, 6, 7, 9, 10)",[0.9023539684543573],0.902354,"(BIRateDelta, BIRateDeltaL2, BIRateChangeL3, B..."
9,"(1, 2, 3, 4, 5, 6, 7, 9, 10)",[0.9104072470254607],0.910407,"(BIRateDelta, BIRateDeltaL2, GDPL1, BIRateChan..."
10,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",[0.913306736532231],0.913307,"(BIRateDelta, BIRateDeltaL2, GDPL1, BIRateChan..."


In [75]:
## model for prods
X = data[['BIRateL4','GDPDelta']]
# y = data['NonWODPD%-KMG']

# Add a constant term to the independent variables
X = sm.add_constant(X)
model = sm.OLS(y_log, X).fit() # Fit the OLS regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             ORDNPL-PKP   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     58.15
Date:                Tue, 01 Jul 2025   Prob (F-statistic):           2.53e-10
Time:                        23:17:17   Log-Likelihood:                 6.1616
No. Observations:                  29   AIC:                            -6.323
Df Residuals:                      26   BIC:                            -2.221
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.9152      0.202    -29.352      0.0